# SoReMo Illinois Interactive Map (Folium)

This notebook builds the first version of the Illinois data center map.

Layers:
- Existing
- Proposed / Under Dev
- Denied (Naperville)
- Clusters

Outputs:
- Interactive HTML map saved to `outputs/maps/illinois_datacenters_map.html`


In [ ]:
# Imports + path setup
import os
import sys
from pathlib import Path
import pandas as pd

# Allow imports from src/ when running notebook
REPO_ROOT = Path.cwd().parents[0]  # notebooks/ -> repo root
sys.path.append(str(REPO_ROOT))

from src.config import IL_SITES_CSV, DEFAULT_OUTPUT_MAP
from src.utils.io import read_csv, ensure_parent_dir
from src.mapping.build_map import build_illinois_map

REPO_ROOT, IL_SITES_CSV, DEFAULT_OUTPUT_MAP


(WindowsPath('c:/Users/Owner/Downloads/SoReMo-S26-people-v-hasty-ai'),
 WindowsPath('C:/Users/Owner/Downloads/SoReMo-S26-people-v-hasty-ai/data/processed/il_sites.csv'),
 WindowsPath('C:/Users/Owner/Downloads/SoReMo-S26-people-v-hasty-ai/outputs/maps/illinois_datacenters_map.html'))

In [12]:
# Load data
sites = read_csv(IL_SITES_CSV)
sites.head()

,site_id,name,layer,operator,city,state,address_or_hint,lat,lon,location_precision,surroundings_snapshot,community_signals,stressors,sources
0,IL-EX-01,Meta DeKalb Data Center,existing,Meta,DeKalb,IL,"Gurler Rd, DeKalb, IL",41.884,-88.75,approx,South edge of DeKalb near major arterials and ...,Expansion framed as jobs + investment by City ...,"Substation or grid upgrades, water use, backup...",https://www.cityofdekalb.com/CivicAlerts.aspx?...
1,IL-EX-02,Stream Data Centers Chicago II,existing,Stream Data Centers,Elk Grove Village,IL,"Elk Grove Village, IL (Stream Chicago II)",42.000,-87.97,approx,Elk Grove positioned as a data center and conn...,Partnerships and utility readiness emphasized,"Grid capacity expansions, generator/mechanical...",https://www.streamdatacenters.com/locations/ch...
2,IL-EX-03,Stream Data Centers Chicago I,existing,Stream Data Centers,Elk Grove Village,IL,"Elk Grove Village, IL (Stream Chicago I)",42.000,-87.97,approx,Same regional cluster dynamics as Chicago II,Public framing focuses on infrastructure readi...,"Cumulative impacts across nearby sites, grid u...",https://www.streamdatacenters.com/news/stream-...
3,IL-EX-04,Equinix CH3,existing,Equinix,Elk Grove Village (Chicago metro),IL,"Elk Grove area, Chicago metro (Equinix CH3)",42.000,-87.97,approx,Part of the Elk Grove data center cluster,"Appears in industry operations context, not co...","Grid and redundancy infrastructure, generator ...",https://www.datacenterdynamics.com/en/news/cbo...
4,IL-EX-05,Digital Realty ORD10 (350 E Cermak),existing,Digital Realty,Chicago,IL,"350 E Cermak Rd, Chicago, IL 60616",41.853,-87.62,exact,South Loop dense urban corridor; historic indu...,Emphasis on interconnection ecosystem and mode...,"Generator testing and emissions, rooftop mecha...",https://www.digitalrealty.com/data-centers/ame...


In [13]:
# Validate schema
required_cols = [
    "site_id",
    "name",
    "layer",  # existing / proposed / denied
    "operator",
    "city",
    "state",
    "address_or_hint",
    "lat",
    "lon",
    "location_precision",
    "surroundings_snapshot",
    "community_signals",
    "stressors",
    "sources"
]

missing = [c for c in required_cols if c not in sites.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

print("Schema looks good.")
print("Row count:", len(sites))
print(sites["layer"].value_counts(dropna=False))


Schema looks good.
Row count: 15
layer
existing    7
proposed    7
denied      1
Name: count, dtype: int64


In [14]:
# Check which rows are missing coordinates (expected for v1)
missing_coords = sites[sites[["lat", "lon"]].isna().any(axis=1)]
print("Sites missing lat/lon:", len(missing_coords))
missing_coords[["site_id", "name", "city", "address_or_hint", "location_precision"]]


Sites missing lat/lon: 0


,site_id,name,city,address_or_hint,location_precision


In [15]:
# Build map (default Illinois view)
m = build_illinois_map(sites)
m

In [16]:
# Export HTML map
ensure_parent_dir(DEFAULT_OUTPUT_MAP)
m.save(str(DEFAULT_OUTPUT_MAP))
print("Saved:", DEFAULT_OUTPUT_MAP)


Saved: C:\Users\Owner\Downloads\SoReMo-S26-people-v-hasty-ai\outputs\maps\illinois_datacenters_map.html


In [17]:
# zoom preset maps
presets = {
    "Chicago core": {"center": [41.864, -87.631], "zoom": 12},
    "Elk Grove cluster": {"center": [42.000, -87.970], "zoom": 12},
    "DeKalb cluster": {"center": [41.865, -88.751], "zoom": 12},
    "Naperville denied": {"center": [41.812, -88.170], "zoom": 13},
}

preset_name = "Elk Grove cluster"
p = presets[preset_name]

m2 = build_illinois_map(sites, center=p["center"], zoom_start=p["zoom"])
m2


In [18]:
summary = (
    sites.groupby(["layer", "city"], dropna=False)
    .size()
    .reset_index(name="count")
    .sort_values(["layer", "count"], ascending=[True, False])
)
summary


,layer,city,count
0,denied,Naperville,1
1,existing,Chicago,3
3,existing,Elk Grove Village,2
2,existing,DeKalb,1
4,existing,Elk Grove Village (Chicago metro),1
6,proposed,Elk Grove Village,2
5,proposed,DeKalb area,1
7,proposed,Hoffman Estates,1
8,proposed,Minooka area,1
9,proposed,Mount Prospect area,1


In [19]:
summary_path = REPO_ROOT / "outputs" / "figures" / "il_site_counts_by_layer_city.csv"
summary_path.parent.mkdir(parents=True, exist_ok=True)
summary.to_csv(summary_path, index=False)
summary_path

WindowsPath('c:/Users/Owner/Downloads/SoReMo-S26-people-v-hasty-ai/outputs/figures/il_site_counts_by_layer_city.csv')